In [ ]:
import os
import sys

os.environ["JAVA_HOME"] = "JDK8"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [ ]:
#Importing the data from the URL source on Air Quality in the UK
import requests
import json
import pandas as pd

data = requests.get("https://maps2.bristol.gov.uk/server2/rest/services/ext/air_quality/MapServer/0/query?outFields=*&where=1%3D1&f=geojson").json()
data = pd.DataFrame(data["features"])
data

In [ ]:
#Creates a function that can extract the columns we needs to perform actions on from the data
def getSector(main_subject):   
    return main_subject

In [ ]:
#Extracts the columns
data["latitude"] = data["geometry"].apply(lambda deets : getSector(main_subject=deets["coordinates"][0]))
data["longitude"] = data["geometry"].apply(lambda deets : getSector(main_subject=deets["coordinates"][1]))
data["Pollutants"] = data["properties"].apply(lambda deets : getSector(main_subject=deets["pollutants"]))
data["Location"] = data["properties"].apply(lambda deets : getSector(main_subject=deets["location"]))
data["LocationType"] = data["properties"].apply(lambda deets : getSector(main_subject=deets["LocationClass"]))
data

In [ ]:
#Creates a funtion that normalizes the latitude and longitude columns 
def normalize(coordinate,min,max):    
    new_coord = (coordinate-min)/(max-min)
    return new_coord

assert normalize(90,-90,90) == 1
assert normalize(-180,-180,180) == 0

In [ ]:
#Applies the normalize funtion to creat 2 new columns 
data["Norm_lat"] = data.latitude.apply(normalize, min=-90,max=90)
data["Norm_lon"] = data.longitude.apply(normalize, min=-180,max=180)
data.head(1)

In [ ]:
#Creates a new dataframe with selected cloumns
selected_columns = ['id', 'Location', 'LocationType','Norm_lat','Norm_lon', 'Pollutants']
new_df = data[selected_columns]
new_df


In [ ]:
#Purs new dataframe into a csv file
new_df.to_csv('ETL_assignment.csv', index=False)  # 'index=False' prevents writing row indices